# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.  

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Задачи проекта
* Посчитать, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитать количество обзоров и среднюю оценку;
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
* Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
* Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### 1. Загрузим данные

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student',
 'pwd': 'Sdf4$2;d-d30pp',
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, 
 'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Загрузим датасет с книгами:

In [3]:
books = pd.io.sql.read_sql(
    '''SELECT * 
       FROM books'''
, con = engine) 
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Загрузим датасет с авторами:

In [4]:
authors = pd.io.sql.read_sql(
    '''SELECT * 
       FROM authors'''
   , con = engine) 
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Загрузим датасет с издателями:

In [5]:
publishers = pd.io.sql.read_sql(
    '''SELECT * 
       FROM publishers'''
   , con = engine) 
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Загрузим датасет с рейтингами:

In [6]:
ratings = pd.io.sql.read_sql(
    '''SELECT * 
       FROM ratings'''
   , con = engine) 
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Загрузим датасет с обзорами:

In [7]:
reviews = pd.io.sql.read_sql(
    '''SELECT * 
       FROM reviews'''
   , con = engine) 
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### 2. Выполним задания

#### 2.1 Посчитаем, сколько книг вышло после 1 января 2000 года;

In [8]:
pd.io.sql.read_sql(
    '''SELECT 
           COUNT(title) as number_of_books 
           
       FROM 
           books
           
       WHERE 
           publication_date >'2000-01-01' '''
    
, con = engine)

,number_of_books
0,819


**Вывод:** после 1 января 2000 года вышло **819** книг


#### 2.2 Для каждой книги посчитайте количество обзоров и среднюю оценку

In [13]:
pd.io.sql.read_sql(
    '''SELECT 
            books.book_id, 
            books.title, 
            COUNT(DISTINCT reviews.review_id) as number_of_reviews,  -- вот тут надо добавить DISTINCT
            round(AVG(ratings.rating::numeric),2) as avg_rating
            
       FROM books
       LEFT JOIN ratings ON books.book_id = ratings.book_id
       LEFT JOIN reviews ON ratings.book_id = reviews.book_id
       
       GROUP BY 
            books.book_id
       
       ORDER BY 
            number_of_reviews DESC
           
       '''
    
, con = engine) 

,book_id,title,number_of_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


**Вывод:** оказалось, что больше всех обзоров вышло для книги **Twilight** (Twilight1 #1) в количестве **7**

#### 2.3 Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [10]:
pd.io.sql.read_sql(
    '''SELECT 
           publishers.publisher, 
           COUNT(books.book_id) as number_of_books
           
       FROM publishers
       INNER JOIN books ON books.publisher_id = publishers.publisher_id
       
       WHERE 
           books.num_pages > 50
           
       GROUP BY 
           publishers.publisher
           
       ORDER BY 
           COUNT(books.book_id) DESC
           
       LIMIT 1'''
, con = engine) 

,publisher,number_of_books
0,Penguin Books,42


**Вывод:** в лидерах оказалось издательство **Penguin Books** с **42** книгами.

#### 2.4 Определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [11]:
pd.io.sql.read_sql(
    '''SELECT
           subq.author,
           round(AVG(subq.avg),2) as avg_rating
                     
       FROM (
           SELECT
               books.title,
               authors.author,
               books.author_id,
               COUNT(ratings.rating) as count,
               AVG(ratings.rating)  as avg
                    
       FROM authors
           INNER JOIN books ON books.author_id = authors.author_id
           INNER JOIN ratings ON books.book_id = ratings.book_id
            
       GROUP BY
           authors.author,
           books.title,
           books.author_id
            
       HAVING 
           COUNT(ratings.rating) > 50) as SUBQ
       
       GROUP BY 
           subq.author
       
       ORDER BY 
           avg_rating DESC
           
       LIMIT 1'''
       
, con = engine) 

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.28


**Вывод:** **J.K. Rowling/Mary GrandPré** оказались успешнее всех по **среднему рейтингу** книг, их результат  **4.28** балла.

#### 2.5 Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
x = pd.io.sql.read_sql(
    '''SELECT 
            round(AVG(subq.rev_count))::integer as avg_reviews
            
       FROM (
           SELECT 
               ratings.username, 
               COUNT(ratings.rating) as rating_count, 
               COUNT(reviews.text) as rev_count
               
           FROM books
           RIGHT JOIN ratings ON ratings.book_id= books.book_id
           LEFT JOIN reviews ON reviews.book_id = ratings.book_id AND reviews.username=ratings.username
           
           GROUP BY 
               ratings.username
               
           HAVING 
               COUNT(ratings.username) > 50) as subq'''
, con = engine)
x

,avg_reviews
0,24


**Вывод:** **24**  - такое оказалось количество **обзоров** от пользователей, которые поставили **50** и больше **оценок**

### Общий вывод

* После **1 января 2000** года вышло **819** книг. 
* Оказалось, что больше всех обзоров вышло для книги **Twilight** (Twilight1 #1) в количестве **1120.**  
* В лидерах оказалось издательство **Penguin Books** с **42** книгами.
* **J.K. Rowling/Mary GrandPré** оказались успешнее всех по **среднему рейтингу** книг, их результат  **4.28** балла.  
* **24** - такое оказалось количество обзоров от пользователей, которые поставили **50** и больше оценок.  